## Task 1

In [2]:
from keras import datasets
import numpy as np
#from keras.models import Sequential
#from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_openml
from numpy.random import default_rng

rng = default_rng()

#mnist = fetch_openml("mnist_784")

#X, y = fetch_openml("mnist_784", return_X_y=True)

#X, y = fetch_openml("mnist_784", version=1, return_X_y=True, as_frame=False, parser="pandas")

#X = X / 255

#X = mnist["data"]
#y = mnist["target"]

#print(X[:5])

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

data = datasets.mnist.load_data()

(X_train, y_train), (X_test, y_test) = data

#train_data = np.reshape(train_data,[784,42000])
#train_label = np.zeros((10,42000))

#print(datasets.mnist.load_data())

X_train = X_train / 255
X_test = X_test / 255

X_train = np.reshape(X_train, [60000, 28 * 28])
X_test = np.reshape(X_test, [10000, 28 * 28])

print(X_train[0])

learning_rate = 0.001
epochs = 10
image_size = 28 * 28 # image size is 28x28 pixels
hidden_layer_size = 100 # chosen size of hidden layer; can be any number
output_layer_size = 10 # becuase of digits from 0 to 9

X_test.shape

[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         

(10000, 784)

In [3]:
class Activation_Layer_ReLU:
    
    def forward_pass(self, x):
        
        return np.maximum(0, x)
    
    def backward_pass(self, x):
        
        return (x > 0) * 1
    

In [4]:
class Activation_Layer_Sigmoid:
    
    def forward_pass(self, x):
        
        return 1 / (1 + np.exp(-x))
    
    def backward_pass(self, x):
        
        return self.forward_pass(x) * (1 - self.forward_pass(x))


In [5]:
class Activation_Layer_Softmax:
    
    def forward_pass(self, x):
        
        exponents = np.exp(x - np.max(x, axis = 1, keepdims = True))
        
        return (exponents) / np.sum(exponents)
    
    def backward_pass(self, x):
        
        softmax = self.softmax(x)
        
        return np.diag(softmax) + (softmax * softmax.reshape(14000, 784))


In [6]:
class Layer_Dense:
    
    def __init__(self, n_inputs, n_neurons):
        
        self.weights = 0.01 * np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros((1, n_neurons))

    def forward_pass(self, x):
        
        return np.dot(x, self.weights) + self.biases
    

In [7]:
class Loss:
    
    def calculate_mean_loss(self, y_pred, y_true):
        
        sample_loss = self.forward_propogation(y_pred, y_true)
        
        mean_loss = np.mean(sample_loss)
        
        return mean_loss
        

In [8]:
class Loss_Cross_Entropy(Loss):
    
    def forward_propogation(self, y_pred, y_true):
        
        samples = len(y_pred)
        
        if len(y_true.shape) == 1:
            
            #print("A:", y_pred.shape, y_true.shape, range(samples))
            
            confidence = y_pred[range(samples), y_true]
        
        elif len(y_true.shape) == 2:
            
            confidence = np.sum(y_pred * y_true, axis = 1)
            
        neg_log_likelihood = -np.log(confidence)
        
        return neg_log_likelihood
        

In [9]:
dense_layer1 = Layer_Dense(X_train.shape[1], epochs)

activation_layer1 = Activation_Layer_ReLU()

dense_layer2 = Layer_Dense(epochs, epochs)

activation_layer2 = Activation_Layer_Sigmoid()

dense_layer3 = Layer_Dense(epochs, epochs)

activation_layer3 = Activation_Layer_Softmax()

loss_function = Loss_Cross_Entropy()


In [10]:
lowest_loss = 9999999

best_dense_layer1_weights = dense_layer1.weights.copy()
best_dense_layer1_biases = dense_layer1.biases.copy()

best_dense_layer2_weights = dense_layer2.weights.copy()
best_dense_layer2_biases = dense_layer2.biases.copy()

best_dense_layer3_weights = dense_layer3.weights.copy()
best_dense_layer3_biases = dense_layer3.biases.copy()


In [11]:
for iteration in range(10000):

    dense_layer1.weights += 0.05 * np.random.randn(X_train.shape[1], epochs)
    dense_layer1.biases += 0.05 * np.random.randn(1, epochs)
    
    dense_layer2.weights += 0.05 * np.random.randn(epochs, epochs)
    dense_layer2.biases += 0.05 * np.random.randn(1, epochs)
    
    dense_layer3.weights += 0.05 * np.random.randn(epochs, epochs)
    dense_layer3.biases += 0.05 * np.random.randn(1, epochs)
    
    dense_layer1_forward = dense_layer1.forward_pass(X_train)
    
    #print(X_train.shape[1])
    #print(dense_layer1_forward.shape)
    
    activation_layer1_forward = activation_layer1.forward_pass(dense_layer1_forward)

    dense_layer2_forward = dense_layer2.forward_pass(activation_layer1_forward)

    activation_layer2_forward = activation_layer2.forward_pass(dense_layer2_forward)

    dense_layer3_forward = dense_layer3.forward_pass(activation_layer2_forward)
    
    activation_layer3_forward = activation_layer3.forward_pass(dense_layer3_forward)
    
    #print(y_test.shape)
    
    #print(activation_layer3_forward)
    
    loss = loss_function.calculate_mean_loss(activation_layer3_forward, y_train)
    
    #print("Loss:", loss)
    
    predictions = np.argmax(activation_layer3_forward, axis = 1)

    accuracy = np.mean(predictions == y_train)

    #print("Accuracy:", accuracy)
    
    if loss < lowest_loss:
        
        #print('New set of weights found, iteration:', iteration, 'loss:', loss, 'acc:', accuracy)
        
        best_dense_layer1_weights = dense_layer1.weights.copy()
        best_dense_layer1_biases = dense_layer1.biases.copy()
        
        best_dense_layer2_weights = dense_layer2.weights.copy()
        best_dense_layer2_biases = dense_layer2.biases.copy()
        
        best_dense_layer3_weights = dense_layer3.weights.copy()
        best_dense_layer3_biases = dense_layer3.biases.copy()
        
        lowest_loss = loss
        
    # Revert weights and biases
    else:
        dense_layer1.weights = best_dense_layer1_weights.copy()
        dense_layer1.biases = best_dense_layer1_biases.copy()
        
        dense_layer2.weights = best_dense_layer2_weights.copy()
        dense_layer2.biases = best_dense_layer2_biases.copy()
        
        dense_layer3.weights = best_dense_layer3_weights.copy()
        dense_layer3.biases = best_dense_layer3_biases.copy()

    if iteration % 1000 == 0:
        
        print("iteration:", iteration, "loss:", loss, "acc:", accuracy)
        

iteration: 0 loss: 13.307504760643193 acc: 0.10218333333333333
iteration: 1000 loss: 12.780883493017255 acc: 0.4464666666666667
iteration: 2000 loss: 12.523787373937408 acc: 0.5032
iteration: 3000 loss: 12.362424288289546 acc: 0.57075
iteration: 4000 loss: 12.20588202737463 acc: 0.6178833333333333
iteration: 5000 loss: 12.144667362049454 acc: 0.6494666666666666
iteration: 6000 loss: 12.112018868449486 acc: 0.66285
iteration: 7000 loss: 12.103830907982243 acc: 0.6597833333333334
iteration: 8000 loss: 12.064912754829617 acc: 0.66475
iteration: 9000 loss: 12.055176015733013 acc: 0.6756833333333333


In [12]:
print("Loss:", loss)
print("Accuracy:", accuracy)

Loss: 12.033241483160662
Accuracy: 0.6847


## -----------------------------------------------------------------------------------------------------------------------------

In [235]:
dense_layer1_forward = dense_layer1.forward_pass(X_train)

#print(X_train.shape[1])
#print(dense_layer1_forward.shape)

activation_layer1_forward = activation_layer1.forward_pass(dense_layer1_forward)

dense_layer2_forward = dense_layer2.forward_pass(activation_layer1_forward)

activation_layer2_forward = activation_layer2.forward_pass(dense_layer2_forward)

dense_layer3_forward = dense_layer3.forward_pass(activation_layer2_forward)

activation_layer3_forward = activation_layer3.forward_pass(dense_layer3_forward)

#print(y_test.shape)

print(activation_layer3_forward)

loss = loss_function.calculate_mean_loss(activation_layer3_forward, y_train)

print("Loss:", loss)

[[1.67590275e-06 1.65152065e-06 1.65721082e-06 ... 1.65715403e-06
  1.67848615e-06 1.67018360e-06]
 [1.67588782e-06 1.65155903e-06 1.65721330e-06 ... 1.65719159e-06
  1.67850166e-06 1.67018850e-06]
 [1.67591281e-06 1.65160950e-06 1.65722753e-06 ... 1.65719984e-06
  1.67855277e-06 1.67021450e-06]
 ...
 [1.67591354e-06 1.65152998e-06 1.65723682e-06 ... 1.65715801e-06
  1.67850356e-06 1.67018524e-06]
 [1.67590307e-06 1.65158943e-06 1.65719749e-06 ... 1.65718721e-06
  1.67852821e-06 1.67022410e-06]
 [1.67589272e-06 1.65154629e-06 1.65720792e-06 ... 1.65716691e-06
  1.67847822e-06 1.67018461e-06]]
Loss: 13.304818431623891


In [236]:
predictions = np.argmax(activation_layer3_forward, axis = 1)

accuracy = np.mean(predictions == y_train)

print("Accuracy:", accuracy)

Accuracy: 0.10218333333333333


## -----------------------------------------------------------------------------------------------------------------------------

In [1]:
class NeuralNetwork:
    
    def __init__(self, no_of_in_nodes, no_of_out_nodes, no_of_hidden_nodes, learning_rate):
        self.no_of_in_nodes = no_of_in_nodes
        self.no_of_out_nodes = no_of_out_nodes
        self.no_of_hidden_nodes = no_of_hidden_nodes
        self.learning_rate = learning_rate 
        self.create_weight_matrices()
        
    
        

In [53]:
def sigmoid(x):
    
    return 1 / (1 + np.exp(-x))

def relu(x):
    
    return np.maximum(0, x)

w1 = rng.random((image_size, hidden_layer_size))
w2 = rng.random((hidden_layer_size, output_layer_size))

print(w1.shape)

for i in range(epochs):
    
    loss = 0.0
    
    for j in range(len(X_train)):
        
        layer_0 = X_train[j]
        
        layer_1 = np.dot(layer_0, w1)
        layer_1 = relu(layer_1)
        
        layer_2 = np.dot(layer_1, w2)
    

(784, 100)
